In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip3 install pytorch-accelerated
# !pip3 install accelerate==0.21.0
!pip3 install datasets

In [ ]:
# Import necessary libraries
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split

from datasets import load_metric
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

In [ ]:
dataset = load_dataset("KaungHtetCho/MedicalQA")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 496792
    })
})

In [ ]:
# def format_instruction(examples):
# 	output_texts = []

# 	for i in range(len(examples['Description'])):
# 		instruction = examples["Description"][i]
# 		input_text = examples["Patient"][i] if 'input' in examples.keys() else ""
# 		response = examples["Doctor"][i]

# 		if len(input_text) > 1:
# 			text = f"""
# Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {instruction}

# ### Input:
# {input_text}

# ### Response:
# {response}
# """.strip()

# 		else:
# 			text = f"""
# Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {instruction}

# ### Response:
# {response}
# """.strip()

# 		output_texts.append(text)

# 	return output_texts

In [ ]:
# formatted_dataset = dataset.map(format_instruction, remove_columns=['Description', 'Patient', 'Doctor'])
# formatted_dataset['train']

In [ ]:
# Split the dataset into training and test data (90% train, 10% test)
# reduce_dataset_split = dataset["train"].train_test_split(test_size=0.01) # make training dataset smaller for testing purpose.
# train_set = reduce_dataset_split['test']

train_test_split = dataset["train"].train_test_split(test_size=0.1)
test_set = train_test_split['test']

# Further split the test set into validation and test sets (50% validation, 50% test)
val_test_split = test_set.train_test_split(test_size=0.5)
validation_set = val_test_split['train']
test_set = val_test_split['test']

# Update dataset to include these splits
dataset["train"] = train_test_split['train']
dataset["validation"] = validation_set
dataset["test"] = test_set

In [ ]:
dataset["train"]

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 447112
})

In [ ]:
# from langchain.memory import ChatMessageHistory

# history = ChatMessageHistory()
# history

In [ ]:
# from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
# memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
# memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
# memory.load_memory_variables({})

In [ ]:
# from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory(return_messages = True)
# memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
# memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
# memory.load_memory_variables({})

In [ ]:
from transformers import AutoModel, AutoTokenizer, pipeline


from transformers import PLBartForConditionalGeneration, PLBartTokenizer

# Load BERT model and tokenizer
model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")
tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="en_XX")

# Define data processing function
def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["Patient"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(batch["Doctor"], padding="max_length", truncation=True, max_length=512)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids
    return batch

# Tokenize and align the data in the dataset
dataset = dataset.map(process_data_to_model_inputs, batched=True)

# pipe = pipeline(
#     task="text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens = 256,
#     model_kwargs = {
#         "temperature" : 0,
#         "repetition_penalty": 1.5
#     }
# )

# llm = HuggingFacePipeline(pipeline=pipe)


Map:   0%|          | 0/447112 [00:00<?, ? examples/s]

Map:   0%|          | 0/24840 [00:00<?, ? examples/s]

Map:   0%|          | 0/24840 [00:00<?, ? examples/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
def collate_fn(batch):
    # Convert list of dicts to dict of lists
    batch = {key: [d[key] for d in batch] for key in batch[0]}

    # Convert list to tensors
    batch["input_ids"] = torch.tensor(batch["input_ids"]).to(device)
    batch["attention_mask"] = torch.tensor(batch["attention_mask"]).to(device)
    batch["labels"] = torch.tensor(batch["labels"]).to(device)
    return batch

In [ ]:
def validate(model, tokenizer, val_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # Disable gradient calculation
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    average_loss = total_loss / len(val_loader)
    return average_loss

In [ ]:
def format_time(seconds):
    """Converts seconds to a string of hours, minutes, and seconds"""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)}h {int(minutes)}m {int(seconds)}s"

In [ ]:
batch_size = 25

# Convert to pytorch tensors and create data loaders
train_loader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset["validation"], batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(dataset["test"], batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
import torch
import time

# Check for GPU availability and move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the training loop
# Define the training loop
num_epoch = 10
prev_val_loss = float('inf')
for epoch in range(num_epoch):  # number of epochs

    start_time = time.time()  # Record the start time of the epoch

    model.train()

    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate the duration of the epoch
    end_time = time.time()
    epoch_duration = end_time - start_time
    formatted_time = format_time(epoch_duration)

    # Compute training and validation loss
    val_loss = validate(model, tokenizer, val_loader)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f} | Validation Loss: {val_loss:.4f} | Duration: {formatted_time}")

    # Save model and tokenizer after we got better val loss.
    if val_loss < prev_val_loss:
      print(f"{val_loss} < {prev_val_loss}. Saving the model.")
      model.save_pretrained(f"./model_BART")
      tokenizer.save_pretrained(f"./model_BART")
      prev_val_loss = val_loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


Epoch 1, Loss: 0.6173 | Validation Loss: 0.4903 | Duration: 4h 15m 51s
0.49032358340993976 < inf. Saving the model.


KeyboardInterrupt: 

In [ ]:
# # Save model and tokenizer after each epoch
# model.save_pretrained(f"./model_BART")
# tokenizer.save_pretrained(f"./model_BART")

In [ ]:
model_directory = "model_BART"
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/"
!cp -r $model_directory $save_path

In [ ]:
# # Copy the model if you want to begin the colab from here.
# model_directory = "model_t5"
# save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/" + model_directory
# !cp -r $save_path .

In [ ]:
# Load the model and tokenizer from a specified epoch
device = "cuda" if torch.cuda.is_available() else "cpu"

model_directory = "./model_BART"  # Adjust the path to match where you've saved the model

load_model = PLBartForConditionalGeneration.from_pretrained(model_directory)
load_model.to(device)
load_tokenizer = PLBartTokenizer.from_pretrained(model_directory)
load_model = load_model.eval()

In [ ]:
# from nltk.translate.bleu_score import corpus_bleu

# def generate_answers(batch):
#     inputs = load_tokenizer(batch["Patient"], return_tensors="pt", padding=True, truncation=True, max_length=512)
#     outputs = load_model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device))
#     batch["model_output"] = load_tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     return batch

# results = test_loader.dataset.map(generate_answers, batched=True, batch_size=8)
# refs = [[x.split()] for x in results["Doctor"]]
# hyp = [x.split() for x in results["model_output"]]

# bleu_score = corpus_bleu(refs, hyp)
# print("BLEU score:", bleu_score)

In [ ]:
def answer_question(question):
    load_model.eval()
    with torch.no_grad():
        input_ids = load_tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
        outputs = load_model.generate(
            input_ids,
            max_length=512,
            temperature=0.4,
            do_sample=True,
            top_k=50
        )
        answer = load_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example
print(answer_question("I am having a fever, and sore throat. What should I do?"))

In [ ]:
answer_question('What is HIV')

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
# save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models"

In [ ]:
# !cp -r $model_directory $save_path